In [13]:
import datetime
import torch
from quantitative_analytics.indices import indices, indexfixingrepository
from quantitative_analytics.marketdata import marketdata, marketdatarepository
from quantitative_analytics.products import products, indexObservation
from quantitative_analytics.models import models
from quantitative_analytics.calculators import calculators, montecarlocalculator
from quantitative_analytics.interpolators import interpolate
from quantitative_analytics.curves import curves
from quantitative_analytics.analytics import matrixanalytics
from quantitative_analytics.indices import indices
from quantitative_analytics.calculators.evolutionGenerators import evolutionGenerators

import numpy as np

In [14]:
torch.set_printoptions(precision=16)

EPSILON = 0.00000001

1. Define some indices

In [15]:
equity = indices.EquityIndex([], "SPX")
equity2 = indices.EquityIndex([], "AAP")

2. Define the option data

In [16]:
observationDates = [datetime.date(year=2020, month=11, day=30),
                    datetime.date(year=2021, month=6, day=30),
                    datetime.date(year=2021, month=12, day=30)]
expiry = datetime.date(year=2021, month=12, day=30)

option_data = {}
option_data['strike'] = torch.tensor(100.0, requires_grad=True)
option_data['expiry'] = expiry
option_data['index'] = equity

# Create a European Option
europeanOption = products.EuropeanOptionProduct(option_data)

option_data['observationDates'] = observationDates

# Create an Asian Option
asianOption = products.AsianOptionProduct(option_data)

option_data['indices'] = [equity, equity2]
asianBasketOption = products.AsianBasketOptionProduct(option_data)

3. Define the model

In [17]:
modelData = {}

modelDate = datetime.date(year=2020, month=12, day=30)
model = models.LognormalModel(modelData, modelDate)

In [28]:
class EvolutionGeneratorLognormalExtension(evolutionGenerators.EvolutionGeneratorMonteCarloBase):
    def __init__(self, data, productData, indexObservations, futureDates,
                 forwardVarianceVector, forwardCovarianceVector):
        super(EvolutionGeneratorLognormalExtension, self).__init__(data, productData, indexObservations)
        self.data = data
        self.productData = productData
        self.numberOfSimulations = data['NumberOfSimulations']
        self.forwardVarianceVector = forwardVarianceVector
        self.forwardCovarianceVector = forwardCovarianceVector
        self.dates = futureDates
        self.indexObservations = indexObservations

    def createStateTensor(self):
        # Simulate this is really where most of the effort is going to be
        n = len(self.dates)

        m = len(self.forwardCovarianceVector[0])
        print(m)

        # Draw random numbers
        z = torch.randn(size=(n,m,self.numberOfSimulations))

        logsamples = torch.zeros(size=(m,self.numberOfSimulations))

        sampleValues = {}

        for i,it in enumerate(self.dates):
            # Need to implement the pseudosquareroot of the matrix
            dW = torch.mm(self.forwardCovarianceVector[i],z[i,:,:])
            for j in range(m):
                dW[j] = dW[j] - self.forwardVarianceVector[i][j]/2.
            logsamples = logsamples + dW

            sampleValues[it] = logsamples

        return sampleValues

    def getValue(self, date, index, stateTensor):
        return self.indexObservations[index][date].getValue(date,stateTensor)

class LognormalModelExtension(models.BaseModel):
    def __init__(self, data, modelDate : datetime.datetime):
        self.data = data
        self.modelDate = modelDate
        self.internalCurves = {}

        # This should come from the the market data

    def forward(self):
        return self.data['forward']

    def volatility(self):
        return self.data['volatility']

    def createCurveFromMarketData(self, marketDataItem : marketdata.BlackVolatilityMarketData):
        dates = marketDataItem.getDates()
        values = marketDataItem.getValues()
        times = self.datesToTimes(dates)

        volatilityInterpolator = interpolate.BaseInterpolator(times, values)

        return curves.BlackVolatilitySurface(volatilityInterpolator)


    def createEvolutionGenerator(self, simulationData, productData : products.productData.ProductDataBase):
        # Derive the timeline from the product
        dates_underlyings = productData.getDatesUnderlyings()
        dates = np.array(list(dates_underlyings.keys()))

        underlyings = []
        historicalDates = []
        futureDates = []

        for it in dates_underlyings.keys():
            for jt in dates_underlyings[it]:
                underlyings.append(jt)
            if it > self.modelDate:
                futureDates.append(it)
            else:
                historicalDates.append(it)

        underlyings = sorted(list(set(underlyings)))

        fwd = {}
        vol_curve = {}

        for it in underlyings:

            # Get spot out of the repository
            spot_md = marketdatarepository.marketDataRepositorySingleton.getMarketData(
                marketdata.MarketDataEquitySpotBase.getClassTag(), it)
            fwd[it] = spot_md.getValue()

            volatilityMarketData = marketdatarepository.marketDataRepositorySingleton.getMarketData(
                marketdata.BlackVolatilityMarketData.getClassTag(), it)
            vol_curve[it] = self.createCurveFromMarketData(volatilityMarketData)


        forwardSqrtCovarianceVector = []
        forwardVarianceVector = []
        times = self.datesToTimes(futureDates)

        n = len(underlyings)

        for it in underlyings:
            print(it.getIndexString())

        # Make this market data
        correlationMatrix = np.identity(n)


        lastCovariance = torch.zeros(size=(n,n))
        lastVariance = torch.zeros(n)


        for tt in times:
            covariance = torch.zeros(size=(n,n))
            variance = torch.zeros(n)

            for i,it in enumerate(underlyings):
                vol_i = vol_curve[it].getVolatility(tt)
                for j,jt in enumerate(underlyings):
                    vol_j = vol_curve[jt].getVolatility(tt)

                    if i < j:
                        correlationIdentifier = marketdata.CorrelationMarketData.createIdentifier(it,jt)
                        correlationMarketData = marketdatarepository.marketDataRepositorySingleton.getMarketData(
                            marketdata.CorrelationMarketData.getClassTag(), correlationIdentifier)
                        correlationMatrix[i][j] = correlationMatrix[j][i] = correlationMarketData.getValue()

                    covariance[i][j] = vol_i*vol_j*correlationMatrix[i][j] * tt
                variance[i] = covariance[i][i]
            forwardVarianceVector.append( variance - lastVariance )
            forwardSqrtCovarianceVector.append(
                matrixanalytics.square_root_symmetric_matrix(covariance - lastCovariance) )
            lastCovariance = covariance
            lastVariance = variance

        indexObservations = {}
        for it in underlyings:
            indexObservations[it] = {}

        # Here we can very easily differentiate future and past
        for it in historicalDates:
            for jt in underlyings:
                # Get spot out of the repository
                value = indexfixingrepository.indexFixingRepositorySingleton.getFixing(jt,it)
                indexObservations[jt][it] = indexObservation.IndexObservationConstant(value)

        for it in futureDates:
            for j,jt in enumerate(underlyings):
                indexObservations[jt][it] = indexObservation.IndexObservationScaledExponential(fwd[jt],j)

        return EvolutionGeneratorLognormalExtension(simulationData, productData, indexObservations,
                                                               futureDates, forwardVarianceVector,
                                                               forwardSqrtCovarianceVector)


if __name__ == '__main__':
    observationDate = datetime.date(year=2021, month=6, day=30)
    expiry = datetime.date(year=2021, month=12, day=30)
    dates = [observationDate, expiry]
    values = torch.tensor([0.2,0.4])

    data = []
    volatilityMarketData = marketdata.BlackVolatilityMarketData(data, dates, values)

    modelData = []
    modelDate = datetime.date(year=2020, month=12, day=30)
    model = LognormalModelExtension(modelData, modelDate)

    volatilityCurve = model.createCurveFromMarketData(volatilityMarketData)

    print(volatilityCurve.getVolatility(0.75))

    times = model.datesToTimes(dates)
    print(times)

tensor(0.2000000029802322)
[0.4986301369863014, 1.0]


In [29]:
modelData = {}

modelDate = datetime.date(year=2020, month=12, day=30)
model_extension = LognormalModelExtension(modelData, modelDate)

4. Define fixings 

In [30]:
indexfixingrepository.indexFixingRepositorySingleton.clear()

fixingDate = datetime.date(year=2020, month=11, day=30)
spot_fixing = torch.tensor(100.0, requires_grad=True)
indexfixingrepository.indexFixingRepositorySingleton.storeFixing(equity, fixingDate, spot_fixing)

spot_fixing2 = torch.tensor(100.0, requires_grad=True)
indexfixingrepository.indexFixingRepositorySingleton.storeFixing(equity2, fixingDate, spot_fixing2)

5. Define market data

In [31]:
marketdatarepository.marketDataRepositorySingleton.clear()

observationDates = [datetime.date(year=2020, month=11, day=30),
                    datetime.date(year=2021, month=6, day=30),
                    datetime.date(year=2021, month=12, day=30)]
expiry = datetime.date(year=2021, month=12, day=30)

forward = torch.tensor([100.0], requires_grad=True)
md = marketdata.MarketDataEquitySpotBase(equity, forward)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(md)

forward2 = torch.tensor([100.0], requires_grad=True)
md2 = marketdata.MarketDataEquitySpotBase(equity2, forward2)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(md2)

volatilityDates = [datetime.date(year=2021, month=6, day=30),
                    datetime.date(year=2021, month=12, day=30)]
volatilityPoint1 = torch.tensor([0.2], requires_grad=True)
volatilityPoint2 = torch.tensor([0.2], requires_grad=True)

volatilityValues = [volatilityPoint1, volatilityPoint2]
volatilityMarketData = marketdata.BlackVolatilityMarketData(equity, volatilityDates, volatilityValues)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(volatilityMarketData)

volatilityPoint3 = torch.tensor([0.3], requires_grad=True)
volatilityPoint4 = torch.tensor([0.3], requires_grad=True)

volatilityValues2 = [volatilityPoint3, volatilityPoint4]
volatilityMarketData2 = marketdata.BlackVolatilityMarketData(equity2, volatilityDates, volatilityValues2)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(volatilityMarketData2)

correlation = torch.tensor([0.1], requires_grad=True)
correlationMarketData = marketdata.CorrelationMarketData(equity2, equity, correlation)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(correlationMarketData)

correlation2 = torch.tensor([0.1], requires_grad=True)
correlationMarketData2 = marketdata.CorrelationMarketData(equity, equity2, correlation2)
marketdatarepository.marketDataRepositorySingleton.storeMarketData(correlationMarketData2)




6. Define the simulation data and run

In [32]:
# Run the Monte-Carlo
simulationData = {}
simulationData['NumberOfSimulations'] = 100000

simulationData['LegValues'] = True
mc = montecarlocalculator.MonteCarloSimulator(simulationData, model_extension, europeanOption)
npvmc = mc.npv()

# Compute first order derivatives
x = [forward]

dxs = []
for it in npvmc:
    dx, = torch.autograd.grad(it, x, create_graph=True, retain_graph=True)
    dxs.append(dx)

ddxs = []
for it in dxs:
    ddx, = torch.autograd.grad(it, x, create_graph=True)
    ddxs.append(ddx)

print(npvmc)
print(dxs)
print(ddxs)

SPX
1
tensor([99.9469070434570312,  7.9197406768798828], grad_fn=<StackBackward>)
[tensor([0.9994690418243408], grad_fn=<SumBackward1>), tensor([0.5370868444442749], grad_fn=<SumBackward1>)]
[tensor([0.], grad_fn=<SumBackward1>), tensor([0.0106064919382334], grad_fn=<SumBackward1>)]
